In [1]:
import os, sys
sys.path.append("..")

In [2]:
from pauli_c import _PauliElement
help(_PauliElement)

Help on class _PauliElement:

class _PauliElement(builtins.object)
 |  Basic Pauli element
 |
 |  Methods defined here:
 |
 |  __eq__(self, value, /)
 |      Return self==value.
 |
 |  __ge__(self, value, /)
 |      Return self>=value.
 |
 |  __gt__(self, value, /)
 |      Return self>value.
 |
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __le__(self, value, /)
 |      Return self<=value.
 |
 |  __lt__(self, value, /)
 |      Return self<value.
 |
 |  __matmul__(self, value, /)
 |      Return self@value.
 |
 |  __mul__(self, value, /)
 |      Return self*value.
 |
 |  __ne__(self, value, /)
 |      Return self!=value.
 |
 |  __repr__(self, /)
 |      Return repr(self).
 |
 |  __rmatmul__(self, value, /)
 |      Return value@self.
 |
 |  __rmul__(self, value, /)
 |      Return value*self.
 |
 |  __str__(self, /)
 |      Return str(self).
 |
 |  commute(...)
 |      Check the commutation relationship between two Paul

In [9]:
n = 10
p00 = _PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p01 = _PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)


In [10]:
p00

_PauliElement(n=10)

In [5]:
p00_.nx, p00_.nz, p00_.n, p00_.f

(3, 6, 10, 1)

In [6]:
p00_.pstr

'IIIIIIIZYX'

In [7]:
p00_.sym_code

(3, 6)

In [8]:
p00_.ij_code

(6, 5)

## Speed comparsion

In [12]:
from numbers import Number
from functools import reduce

import numpy as np


class PauliElement(_PauliElement):
    _coef_factor = {0: 1, 1: -1j, 2: -1, 3: 1j}
    def __str__(self):
        return f"{self.pstr}"
    def __repr__(self):
        return f"Pauli(xz=({self.nx}, {self.nz}), {self.pstr}, n={self.n}, weight={self.weight})"
    def __rmul__(self, other:Number):
        return PauliElement(self.nx, self.nz, self.n, other*self.weight)
    def __matmul__(self, other):
        _pr= super().__matmul__(other)
        return PauliElement(_pr.nx, _pr.nz, _pr.n, _pr.weight, _pr.f, set_phase=False)
    def to_pennylane(self, words=False):
        from pennylane.pauli import string_to_pauli_word
        from pennylane import I, PauliX, PauliY, PauliZ
        if words:
            map_dict = {i:i for i in (range(self.n))}
            terms = string_to_pauli_word(self.string, wire_map=map_dict)
        else:
            ops = {
                "I": lambda i: I(i),
                "X": lambda i: PauliX(i),
                "Y": lambda i: PauliY(i),
                "Z": lambda i: PauliZ(i),
                   }
            opers = [ops[s](i) for i, s in enumerate(reversed(self.string))]
            terms =reduce(lambda x, y: x@y, opers)
        return self.coef, terms
    
    def to_qiskit(self):
        from qiskit.quantum_info import Pauli as qiskit_Pauli
        x_array = np.flip(np.array([int(b) for b in format(self.nx, f"0{self.n}b")], dtype=np.uint))
        z_array = np.flip(np.array([int(b) for b in format(self.nz, f"0{self.n}b")], dtype=np.uint))
        return self.weight, qiskit_Pauli((z_array, x_array))

## Compare with Qiskit

In [13]:
p00 = PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p01 = PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)
w00, q_p00 = p00.to_qiskit()
w01, q_p01 = p01.to_qiskit()


In [14]:
q_p00.compose(q_p01)

Pauli('IIIIIIIXYZ')

In [18]:
p02_= p00_@p01_
p02_.pstr

'IIIIIIIXYZ'

## Speed

### Qiskit

In [32]:
%%timeit
q_p00.compose(q_p01)

5.92 μs ± 194 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### OptTrot

In [31]:
%%timeit
p00_@p01_

672 ns ± 3.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Numpy binary

In [28]:
import numpy as np

nx1= np.random.randint(0, 2, size=n, dtype=bool)
nz1= np.random.randint(0, 2, size=n, dtype=bool)
nx2= np.random.randint(0, 2, size=n, dtype=bool)
nz2= np.random.randint(0, 2, size=n, dtype=bool)
f1 = 3
f2 = 4


In [33]:
%%timeit
nx3 = np.bitwise_xor(nx1, nx2)
nz3 = np.bitwise_xor(nz1, nz2)
f3 = f1+f2 + 2*np.bitwise_and(nx1, nz2).astype(int).sum() -  np.bitwise_and(nx3, nz3).astype(int).sum()

3.12 μs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
